In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

/home/binil/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Simple Reggression Approach

In [3]:
# Placeholders
x=tf.placeholder(tf.float32,shape=[None,784])

In [4]:
# Variables
W=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))

In [5]:
# Create Graph Operations
y=tf.matmul(x,W)+b

In [6]:
# Loss Function
y_true=tf.placeholder(tf.float32,[None,10])

In [7]:
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
# Optimizer
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.5)
train=optimizer.minimize(cross_entropy)

In [9]:
# Create Session
init=tf.global_variables_initializer()

<function tensorflow.contrib.layers.python.layers.initializers.variance_scaling_initializer.<locals>._initializer>

In [10]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(1000):
        batch_x,batch_y=mnist.train.next_batch(100)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y})
    
    # Evaluate
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_true,1))
    acc=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels}))

0.9202


# Convolutional Neural Networks

In [73]:
def init_weights(shape):
#     init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    initializer = tf.contrib.layers.xavier_initializer()
    init_random_dist = tf.Variable(initializer(shape))
    
    return tf.Variable(init_random_dist)

In [74]:
# temp = tf.get_variable("temp", shape=[10, 1],
#            initializer=tf.contrib.layers.xavier_initializer())
# print(tf.Session().run(temp))

initializer = tf.contrib.layers.xavier_initializer()
w1 = tf.Variable(initializer([10,1]))
# b1 = tf.Variable(initializer([10,1]))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w1))


[[ 0.40526384]
 [-0.5236355 ]
 [-0.08122778]
 [-0.30332157]
 [-0.21654946]
 [-0.47473016]
 [ 0.16609913]
 [-0.4481035 ]
 [-0.6629523 ]
 [-0.28438184]]


In [75]:
  
import tensorflow as tf
tf.reset_default_graph() 
a = tf.get_variable("a",shape=[4, 4], initializer=tf.contrib.layers.xavier_initializer())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(a))

[[ 0.303142    0.7538673   0.70315665  0.2623964 ]
 [-0.45814687 -0.4293917   0.17941982  0.59578294]
 [-0.34716862  0.30839664 -0.49963894  0.22897738]
 [ 0.09103262  0.09150815 -0.80591035  0.14043456]]


In [76]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

Create a 2D convolution using builtin conv2d from TF. From those docs:

Computes a 2-D convolution given 4-D input and filter tensors.

Given an input tensor of shape [batch, in_height, in_width, in_channels] and a filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels], this op performs the following:

    Flattens the filter to a 2-D matrix with shape [filter_height * filter_width * in_channels, output_channels].
    Extracts image patches from the input tensor to form a virtual tensor of shape [batch, out_height, out_width, filter_height * filter_width * in_channels].
    For each patch, right-multiplies the filter matrix and the image patch vector.



In [77]:
def conv2d(x, W):    
    # x--> [Batch,Height,Width,Channels]
    # W--> [Filter H, Filter W, Channels In,  Channels Out]
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

Create a max pooling layer, again using built in TF functions:

Performs the max pooling on the input.

Args:
  value: A 4-D `Tensor` with shape `[batch, height, width, channels]` and
    type `tf.float32`.
  ksize: A list of ints that has length >= 4.  The size of the window for
    each dimension of the input tensor.
  strides: A list of ints that has length >= 4.  The stride of the sliding
    window for each dimension of the input tensor.
  padding: A string, either `'VALID'` or `'SAME'`. 


In [78]:
def max_pool_2by2(x):
    # x--> [Batch,Height,Width,Channels]
    # ksize -->
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [79]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

Fully Connected Layer

In [80]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [81]:
# Placeholders
x=tf.placeholder(tf.float32,[None,784])
y_true=tf.placeholder(tf.float32,[None,10])

In [82]:
# Layers
x_image=tf.reshape(x,[-1,28,28,1])

In [83]:
convo_1=convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling=max_pool_2by2(convo_1)

In [84]:
convo_2=convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling=max_pool_2by2(convo_2)

In [85]:
convo_2_flat=tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one=normal_full_layer(convo_2_flat,1024)

In [86]:
# Dropout
hold_prob=tf.placeholder(tf.float32)
full_one_dropout=tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [87]:
y_pred=normal_full_layer(full_one_dropout,10)

In [88]:
# Loss Function
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [89]:
# Optimizer
optimizer=tf.train.AdamOptimizer(learning_rate=0.001)
train=optimizer.minimize(cross_entropy)

In [90]:
init=tf.global_variables_initializer()

In [91]:
mnist.train.num_examples

55000

In [92]:
steps=5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        batch_x,batch_y=mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:.5})
        
        if i%10 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')
    

Currently on step 0
Accuracy is:
0.1365


Currently on step 10
Accuracy is:
0.0982


Currently on step 20
Accuracy is:
0.1521


Currently on step 30
Accuracy is:
0.2762


Currently on step 40
Accuracy is:
0.6563


Currently on step 50
Accuracy is:
0.7918


Currently on step 60
Accuracy is:
0.7754


Currently on step 70
Accuracy is:
0.838


Currently on step 80
Accuracy is:
0.8291


Currently on step 90
Accuracy is:
0.8476


Currently on step 100
Accuracy is:
0.8465


Currently on step 110
Accuracy is:
0.8525


Currently on step 120
Accuracy is:
0.8666


Currently on step 130
Accuracy is:
0.8838


Currently on step 140
Accuracy is:


KeyboardInterrupt: 